In [3]:
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
token = user_secrets.get_secret("github_repos_wildcard")

In [4]:
repo_url = f"https://{token}@github.com/gaserSami/panther.git"
branch = "autotuner"

In [5]:
!git clone -b {branch} {repo_url}

Cloning into 'panther'...
remote: Enumerating objects: 1301, done.
remote: Counting objects: 100% (275/275), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 1301 (delta 249), reused 232 (delta 231), pack-reused 1026 (from 1)
Receiving objects: 100% (1301/1301), 27.78 MiB | 18.53 MiB/s, done.
Resolving deltas: 100% (839/839), done.


In [6]:
# # First uninstall existing torch, torchvision, torchaudio
# !pip uninstall -y torch torchvision torchaudio

# # Install the specified versions from PyTorch's official CUDA 12.4 wheels
# !pip install torch==2.6.0+cu124 torchvision==0.21.0+cu124 torchaudio==2.6.0+cu124 --index-url https://download.pytorch.org/whl/cu124

In [7]:
!mv panther Panther

In [8]:
!ls

Panther


In [9]:
# %%writefile /kaggle/working/Panther/pawX/setup.py
# from setuptools import setup
# from torch.utils.cpp_extension import BuildExtension, CUDAExtension

# setup(
#     name="pawX",
#     ext_modules=[
#         CUDAExtension(
#             name="pawX",
#             sources=[
#                 "skops.cpp",
#                 "bindings.cpp",
#                 "linear.cpp",
#                 "linear_cuda.cu",
#                 "cqrrpt.cpp",
#                 "rsvd.cpp",
#                 "attention.cpp",
#                 "conv2d.cpp"
#             ],
#             # Use system includes and libraries
#             include_dirs=["/usr/include/x86_64-linux-gnu"],
#             library_dirs=[],
#             libraries=["openblas"],
#             extra_compile_args={"cxx": ["-O2", "-fopenmp"], "nvcc": ["-O2"]},
#             extra_link_args=["-llapacke", "-lopenblas"]
#         )
#     ],
#     cmdclass={"build_ext": BuildExtension},
# )

In [10]:
# !sudo apt-get install liblapacke-dev

In [11]:
# !cd /kaggle/working/Panther/pawX; python setup.py install
# !cd /kaggle/working/Panther/pawX; pip install --no-build-isolation -e .

In [12]:
import torch

print(torch.__version__)
import triton

print(triton.__version__)

2.5.1+cu124
3.1.0


In [13]:
import os

os.chdir("/kaggle/working/Panther")

In [14]:
!pwd

/kaggle/working/Panther


In [17]:
!pip install botorch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 738.3/738.3 kB 11.3 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.7/277.7 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.3/176.3 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 72.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━

In [19]:
import torch
from transformers import BertForMaskedLM
from panther.tuner.SkAutoTuner.ModelVisualizer import ModelVisualizer
import torch.nn as nn


def print_linear_layer_params(model):
    """
    Print parameters of all Linear layers in the BERT model.
    This provides detailed information about each Linear layer's configuration.
    """
    print("\n===== Linear Layer Parameters =====")
    for name, module in model.named_modules():
        if isinstance(module, nn.Linear):
            print(f"Layer: {name}")
            print(f"  - in_features: {module.in_features}")
            print(f"  - out_features: {module.out_features}")
            print(f"  - bias: {module.bias is not None}")
            # Print shape information
            weight_shape = module.weight.shape
            print(f"  - weight shape: {weight_shape}")
            if module.bias is not None:
                bias_shape = module.bias.shape
                print(f"  - bias shape: {bias_shape}")
            print("----------------------------------------")


def visualize_bert_model():
    """
    Load a BERT model and create an interactive visualization of its structure.
    This function demonstrates how to use ModelVisualizer with a BERT model.
    """
    print("Loading BERT model for visualization...")

    # Load BERT model
    model = BertForMaskedLM.from_pretrained("bert-base-uncased")

    # Set model to evaluation mode
    model.eval()

    # Print basic model information
    total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Model: BERT base uncased")
    print(f"Total trainable parameters: {total_params:,}")

    # Print details of all linear layers in the model
    print_linear_layer_params(model)

    # Print model tree structure in console
    print("\nModel tree structure:")
    ModelVisualizer.print_module_tree(model)

    # Create an interactive visualization
    print("\nCreating interactive visualization...")
    output_path = ModelVisualizer.create_interactive_visualization(
        model=model, output_path="bert_model_visualization.html", open_browser=True
    )

    print(f"\nVisualization saved to: {output_path}")
    print("The visualization should have opened in your default web browser.")
    print("If not, you can open it manually by navigating to the file.")

    # Additional information about using the visualization
    print("\nUsing the visualization:")
    print("- Hover over modules to see details like parameter counts")
    print("- Click on modules to expand/collapse their children")
    print("- Use the search box to find specific modules")
    print("- The interactive interface allows exploring the entire model structure")


if __name__ == "__main__":
    # Call the visualization function
    visualize_bert_model()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Loading BERT model for visualization...
Model: BERT base uncased
Total trainable parameters: 109,514,298

===== Linear Layer Parameters =====
Layer: bert.encoder.layer.0.attention.self.query
  - in_features: 768
  - out_features: 768
  - bias: True
  - weight shape: torch.Size([768, 768])
  - bias shape: torch.Size([768])
----------------------------------------
Layer: bert.encoder.layer.0.attention.self.key
  - in_features: 768
  - out_features: 768
  - bias: True
  - weight shape: torch.Size([768, 768])
  - bias shape: torch.Size([768])
----------------------------------------
Layer: bert.encoder.layer.0.attention.self.value
  - in_features: 768
  - out_features: 768
  - bias: True
  - weight shape: torch.Size([768, 768])
  - bias shape: torch.Size([768])
----------------------------------------
Layer: bert.encoder.layer.0.attention.output.dense
  - in_features: 768
  - out_features: 768
  - bias: True
  - weight shape: torch.Size([768, 768])
  - bias shape: torch.Size([768])
-------